In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns

import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost.sklearn import XGBClassifier
#from sklearn import cross_validation, metrics   #Additional     scklearn functions
#from sklearn.grid_search import GridSearchCV   #Perforing grid search
from scipy import stats
file= './train.csv'
train=pd.read_csv(file)
file1='./test.csv'
test1=pd.read_csv(file1)


In [42]:
train.rename(columns={'时间':'time', '小区名':'community', '小区房屋出租数量':'house_num', '楼层':'floor', '总楼层':'total_floor', '房屋面积':'area', '房屋朝向':'orientation', '居住状态':'status', '卧室数量':'bedroom_num',
       '厅的数量':'livingroom_num', '卫的数量':'bathroom_num', '出租方式':'method', '区':'Urban', '位置':'location', '地铁线路':'underground', '地铁站点':'underground_station', '距离':'distance', '装修情况':'Decoration', '月租金':'price'},inplace=True)
test1.rename(columns={'时间':'time', '小区名':'community', '小区房屋出租数量':'house_num', '楼层':'floor', '总楼层':'total_floor', '房屋面积':'area', '房屋朝向':'orientation', '居住状态':'status', '卧室数量':'bedroom_num',
       '厅的数量':'livingroom_num', '卫的数量':'bathroom_num', '出租方式':'method', '区':'Urban', '位置':'location', '地铁线路':'underground', '地铁站点':'underground_station', '距离':'distance', '装修情况':'Decoration', 'id':'id'},inplace=True)

In [47]:
for d in ['house_num','Urban','location']:
    train[d].fillna(train[d].mean(), inplace=True)
for d in ['status','Decoration','method','distance','underground','underground_station']:
    train[d].fillna(0, inplace=True)
    
for d in ['house_num','Urban','location']:
    test1[d].fillna(test1[d].mean(), inplace=True)
for d in ['status','Decoration','method','distance','underground','underground_station']:
    test1[d].fillna(0, inplace=True)
test1['orientation_E'].fillna(test1['orientation_E'].mean(), inplace=True) 

,time,community,house_num,floor,total_floor,area,status,bedroom_num,livingroom_num,bathroom_num,method,Urban,location,underground,underground_station,distance,Decoration,price
count,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000
mean,2.115229,3224.116562,0.123519,0.955449,0.408711,0.013139,0.279232,2.236635,1.299625,1.223818,0.110991,7.903892,67.935265,1.533930,26.847903,0.257395,0.337704,7.949313
std,0.786980,2023.073726,0.133252,0.851511,0.183100,0.008104,0.853629,0.896961,0.613169,0.487234,0.314121,4.026602,43.527326,1.924763,37.431145,0.322763,1.213767,6.310609
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1388.000000,0.039062,0.000000,0.290909,0.009268,0.000000,2.000000,1.000000,1.000000,0.000000,4.000000,33.000000,0.000000,0.000000,0.000000,0.000000,4.923599
50%,2.000000,3086.000000,0.082031,1.000000,0.418182,0.012910,0.000000,2.000000,1.000000,1.000000,0.000000,9.000000,61.000000,0.000000,0.000000,0.000000,0.000000,6.621392
75%,3.000000,5199.000000,0.160156,2.000000,0.563636,0.014896,0.000000,3.000000,2.000000,1.000000,0.000000,11.000000,103.000000,3.000000,57.000000,0.535000,0.000000,8.998302
max,3.000000,6627.000000,1.000000,2.000000,1.000000,1.000000,3.000000,11.000000,8.000000,8.000000,1.000000,14.000000,152.000000,5.000000,119.000000,1.000000,6.000000,100.000000


In [32]:
id1=test1.id
test1= test1.drop(['id'],axis=1)

In [26]:
quantity = [attr for attr in train.columns if train.dtypes[attr] != 'object']  # 数值变量集合
quality = [attr for attr in train.columns if train.dtypes[attr] == 'object']  # 类型变量集合


In [27]:
def encode(frame,test, feature):
    '''
    对所有类型变量，依照各个类型变量的不同取值对应的样本集内房价的均值，按照房价均值高低
    对此变量的当前取值确定其相对数值1,2,3,4等等，相当于对类型变量赋值使其成为连续变量。
    此方法采用了与One-Hot编码不同的方法来处理离散数据，值得学习
    注意：此函数会直接在原frame的DataFrame内创建新的一列来存放feature编码后的值。
    '''
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    ordering['price_mean'] = frame[[feature, 'price']].groupby(feature).mean()['price']
    # 上述 groupby()操作可以将某一feature下同一取值的数据整个到一起，结合mean()可以直接得到该特征不同取值的房价均值
    ordering = ordering.sort_values('price_mean')
    ordering['order'] = range(1, ordering.shape[0]+1)
    ordering = ordering['order'].to_dict()
    for attr_v, score in ordering.items():
        # e.g. qualitative[2]: {'Grvl': 1, 'MISSING': 3, 'Pave': 2}
        frame.loc[frame[feature] == attr_v, feature+'_E'] = score
        test1.loc[test1[feature] == attr_v, feature+'_E'] = score

quality_encoded = []
# 由于qualitative集合中包含了非数值型变量和伪数值型变量（多为评分、等级等，其取值为1,2,3,4等等）两类
# 因此只需要对非数值型变量进行encode()处理。
# 如果采用One-Hot编码，则整个qualitative的特征都要进行pd,get_dummies()处理
for q in quality:
    encode(train,test1, q)
    quality_encoded.append(q+'_E')
train.drop(quality, axis=1, inplace=True)
test1.drop(quality, axis=1, inplace=True) # 离散变量已经有了编码后的新变量，因此删去原变量
# df_tr.shape = (1460, 80)
print(quality_encoded, '\n{} qualitative attributes have been encoded.'.format(len(quality_encoded)))




DataError: No numeric types to aggregate

In [6]:
#train,test=train_test_split(train,test_size=0.001)
y = train.price
X = train.drop(['price'],axis=1)
#val_y = test.price
#val_X = test.drop(['price'],axis=1)


#xgb矩阵赋值
#xgb_val = xgb.DMatrix(val_X,label=val_y)
xgb_train = xgb.DMatrix(X, label=y)

In [9]:
#xgboost训练
watchlist = [(xgb_train,'train')]#,(xgb_val,'val')]
params = {'objective':'reg:linear','eval_metric':'rmse','max_depth':6,'min_child_weight':5 }

bst = xgb.train(params,dtrain=xgb_train,num_boost_round=7000,evals = watchlist,early_stopping_rounds=100)
# make prediction
#p = bst.predict(xgb_val)
#p=np.expm1(p) 对数处理未完善
#print(mean_squared_error(val_y,p)**0.5)

[10:04:14] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[0]	train-rmse:7.40189
Will train until train-rmse hasn't improved in 100 rounds.
[10:04:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[1]	train-rmse:5.83758
[10:04:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[2]	train-rmse:4.84187
[10:04:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[3]	train-rmse:4.20387
[10:04:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[4]	train-rmse:3.81109
[10:04:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater

[10:04:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=6
[47]	train-rmse:2.35924
[10:04:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=6
[48]	train-rmse:2.34825
[10:04:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[49]	train-rmse:2.33602
[10:04:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[50]	train-rmse:2.31456
[10:04:29] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=6
[51]	train-rmse:2.31045
[10:04:29] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 102 extra node

[10:04:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[94]	train-rmse:2.01472
[10:04:40] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=6
[95]	train-rmse:2.01124
[10:04:40] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=6
[96]	train-rmse:2.01046
[10:04:40] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=6
[97]	train-rmse:2.00882
[10:04:40] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[98]	train-rmse:2.00479
[10:04:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes,

[10:04:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 0 pruned nodes, max_depth=6
[141]	train-rmse:1.85071
[10:04:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=6
[142]	train-rmse:1.84895
[10:04:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[143]	train-rmse:1.83895
[10:04:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=6
[144]	train-rmse:1.83343
[10:04:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[145]	train-rmse:1.83089
[10:04:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 92 extra no

[10:05:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=6
[188]	train-rmse:1.70904
[10:05:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=6
[189]	train-rmse:1.70659
[10:05:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=6
[190]	train-rmse:1.70525
[10:05:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[191]	train-rmse:1.70284
[10:05:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[192]	train-rmse:1.70184
[10:05:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 110 extra

...

[10:30:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[6779]	train-rmse:0.507275
[10:30:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[6780]	train-rmse:0.507232
[10:30:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=6
[6781]	train-rmse:0.507165
[10:30:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[6782]	train-rmse:0.507149
[10:30:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 0 pruned nodes, max_depth=6
[6783]	train-rmse:0.507122
[10:30:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 

[10:31:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[6825]	train-rmse:0.5058
[10:31:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=6
[6826]	train-rmse:0.505761
[10:31:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=6
[6827]	train-rmse:0.50575
[10:31:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=6
[6828]	train-rmse:0.505748
[10:31:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 0 pruned nodes, max_depth=6
[6829]	train-rmse:0.50574
[10:31:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 68 ext

[10:31:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=6
[6871]	train-rmse:0.504407
[10:31:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=6
[6872]	train-rmse:0.504363
[10:31:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=6
[6873]	train-rmse:0.50435
[10:31:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[6874]	train-rmse:0.504313
[10:31:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[6875]	train-rmse:0.504272
[10:31:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1

[10:31:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=6
[6917]	train-rmse:0.503342
[10:31:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=6
[6918]	train-rmse:0.503341
[10:31:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 0 pruned nodes, max_depth=6
[6919]	train-rmse:0.503337
[10:31:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=6
[6920]	train-rmse:0.503317
[10:31:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[6921]	train-rmse:0.503293
[10:31:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1

[10:31:48] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=6
[6963]	train-rmse:0.50238
[10:31:49] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 0 pruned nodes, max_depth=6
[6964]	train-rmse:0.502369
[10:31:49] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 0 pruned nodes, max_depth=6
[6965]	train-rmse:0.502358
[10:31:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[6966]	train-rmse:0.502263
[10:31:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=6
[6967]	train-rmse:0.502256
[10:31:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 11

In [10]:
#xgb矩阵赋值并写try.csv文件
xgb_val2 = xgb.DMatrix(test1)
p=bst.predict(xgb_val2)
id1['price']=p

sp = pd.DataFrame()
sp['id']=id1
sp.drop(sp.index[-1],inplace=True)
sp['price']=p
sp.to_csv('try.csv',index=False)
d=pd.read_csv('try.csv')
print(d)

          id      price
0          1   4.406598
1          2   6.274939
2          3  13.076798
3          4   6.149084
4          5   4.977569
5          6   7.040171
6          7  10.723002
7          8   3.733161
8          9   8.149872
9         10   5.846987
10        11   5.266849
11        12   9.000317
12        13   3.855629
13        14   7.345072
14        15   6.901557
15        16  13.139728
16        17   7.074043
17        18   5.403367
18        19   7.626959
19        20   5.227860
20        21   4.190912
21        22   5.351640
22        23   8.402939
23        24   6.455352
24        25   8.407480
25        26   5.267507
26        27   4.535012
27        28   2.952174
28        29   7.774681
29        30   3.541676
...      ...        ...
56249  57970   3.889610
56250  57971   6.174589
56251  57973   6.837356
56252  57974   7.844952
56253  57975   8.259843
56254  57976  13.134116
56255  57977   6.360735
56256  57978   5.054283
56257  57979   4.378596
56258  57980   9

In [ ]:
print('hhhh')